In [ ]:
!pip install python-binance

In [2]:
api_key = "ncaGKCQNu9KS8kEwF3Jri37YjsOkKCfLGiKZcq87iThEU3QZIWdxOyeQLUvOCPpG"
api_secret = "tvSQ36Idx7W1QwiMx5ycduhziLQZaHUbhnHZGhqlSRaF6VrkP8QqsLY3LCidthdJ"

In [3]:
from binance import Client
import pandas as pd

client = Client(api_key, api_secret)

In [ ]:
client.get_account()

In [ ]:
#datastream via websocket - will not be covered in this one

In [4]:
import datetime

def getminutedata(symbol, interval, lookback):
  frame = pd.DataFrame(client.get_historical_klines(symbol, interval, lookback + ' min ago KST'))
  frame = frame.iloc[:, :6]
  frame.columns  = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
  frame = frame.set_index('Time')
  frame.index = pd.to_datetime(frame.index, unit='ms')
  frame = frame.astype(float)
  time_change = datetime.timedelta(hours=9)
  frame.index = frame.index + time_change
  return frame

In [5]:
test = getminutedata('BTCUSDT', '1D', '30')
test
#test.Open.plot()

BinanceAPIException: ignored

In [ ]:
#buy if asset fell by more than 0.2% within the last 30 min
#sell if asset rises by more than 0.15% or falls further by 0.15%

In [ ]:
def strategytest(symbol, qty, entried=False):
  df = getminutedata(symbol, '1m', '30m')
  cumulret = (df.Open.pct_change() + 1).cumprod() - 1
  if not entried:
      if cumulret[-1] < -0.002:
        order = client.create_order(symbol=symbol, side = 'BUY', type='MARKET', quantity=qty)
        print(order)
        entried = True
      else:
        print('No Trade has been executed')
  if entried:
    while True:
      df = getminutedata(symbol, '1m', '30m')
      sincebuy = df.loc[df.index > pd.to_datetime(order['transactTime'], unit = 'ms')]
      if len(sincebuy) > 0:
        sincebuyret = (sincebuy.Open.pct_change() + 1).cumprod() - 1
        if sincebuyret[-1] > 0.0015 or sincebuyret[-1] < -0.0015:
          order = client.create_order(symbol=symbol, side = 'SELL', type='MARKET', quantity=qty)
          print(order)
          break
    